In [ ]:
import pandas as pd 
df=pd.read_excel(r"Y:\Admin - General\Finance\Power BI FIle\Sales Report_New\SAP Extract\03 March\Data.xlsx")
material_df=pd.read_excel(r"C:\Users\141823\Downloads\Material Master Data _Etex AU 5500.xlsx",sheet_name=0,header=None)
customer_df=pd.read_excel(r"C:\Users\141823\Downloads\Customer Master Data _Etex.XLSX", sheet_name='Customer Masterdata')


# Set the second row (index 1) as the column headers
material_df.columns = material_df.iloc[1]

# Drop the first two rows (original headers + new headers row)
material_df = material_df.iloc[2:].reset_index(drop=True)
material_df=material_df[['Material','P&L Classification']].rename(columns={'Material': 'Product', 'P&L Classification': 'PL'})
material_df=material_df.drop_duplicates(subset=['Product','PL'])
customer_df=customer_df[['Customer','Region']].rename(columns={'Region': 'State'})
customer_df=customer_df.drop_duplicates(subset=['Customer', 'State'])

In [3]:
merged_df = df.merge(customer_df, on='Customer', how='left')
# Ensure 'Posting date' is in datetime format
merged_df['Posting date'] = pd.to_datetime(merged_df['Posting date'], errors='coerce')

# Format it as 'day/month/year' (e.g., 10/03/2025)
merged_df['Posting date'] = merged_df['Posting date'].dt.strftime('%d/%m/%Y')

#merged_df.to_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\KE28.xlsx",index=False)

In [5]:
group_df=merged_df
group_df['rev']=merged_df['Net Price']+merged_df['Cat/List price']-merged_df['Comm. Discount 1&2']-merged_df['Comm. Discount 3']-merged_df['Collection Discount']
# Group by 'Product' and sum 'rev'
group_df = group_df.groupby(['Product'], as_index=False).agg({'rev': 'sum'})

# Merge with material_df to bring additional details for each Product
group_df = group_df.merge(material_df, on='Product', how='left')
group_df.head()

,Product,rev,PL
0,49332,80.01,Manufactured - Fire Protection Products
1,78230,880.90,Manufactured - Collar
2,78253,25698.90,Manufactured - Collar
3,78254,6662.19,Manufactured - Collar
4,78255,9085.59,Manufactured - Collar


In [6]:
pl_df=group_df.groupby(['PL'], as_index=False).agg({'rev': 'sum'})


In [7]:
import pandas as pd

# Define the output Excel file path
output_file = r"C:\Users\141823\OneDrive - Etex Group\Desktop\KE28.xlsx"

# Use ExcelWriter to write multiple DataFrames to different sheets
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    merged_df.to_excel(writer, sheet_name='KE24_2025', index=False)
    group_df.to_excel(writer, sheet_name='Product Summary', index=False)
    pl_df.to_excel(writer, sheet_name='PL Summary', index=False)